<a href="https://colab.research.google.com/github/AhmedBenAbdessalam/MNIST_simple_model/blob/main/MNIST_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [118]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [119]:
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

In [120]:
mnist_info.features

FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})

In [121]:
mnist_train , mnist_test = mnist_dataset['train'] ,mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples , tf.int64)


In [122]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples , tf.int64)

In [123]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [124]:
scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_train.map(scale)

BUFFER_SIZE = 10000

shuffed_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffed_train_and_validation_data.take(num_validation_samples)
train_data = shuffed_train_and_validation_data.skip(num_validation_samples)

In [125]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs , validation_targets = next(iter(validation_data))

##Model

In [126]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(200,activation='relu'),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dense(100,activation='relu'),
                             tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [128]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' , metrics=['accuracy'] )

###Training

In [129]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 10s - loss: 0.2895 - accuracy: 0.9165 - val_loss: 0.1364 - val_accuracy: 0.9592 - 10s/epoch - 19ms/step
Epoch 2/5
540/540 - 5s - loss: 0.1099 - accuracy: 0.9666 - val_loss: 0.1012 - val_accuracy: 0.9685 - 5s/epoch - 9ms/step
Epoch 3/5
540/540 - 5s - loss: 0.0760 - accuracy: 0.9763 - val_loss: 0.0691 - val_accuracy: 0.9792 - 5s/epoch - 9ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0547 - accuracy: 0.9824 - val_loss: 0.0625 - val_accuracy: 0.9797 - 5s/epoch - 9ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0441 - accuracy: 0.9864 - val_loss: 0.0455 - val_accuracy: 0.9852 - 5s/epoch - 10ms/step


## Test the model

In [130]:
test_loss, test_accuracy = model.evaluate(test_data)

6/6 [==============================] - 2s 350ms/step - loss: 0.0297 - accuracy: 0.9908


In [131]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

Test loss: 0.03. Test accuracy: 99.08%
